#ChatBot Development (Part 2)

## This is a basic notebook with everything to design and experipment with a ChatBot. You have the ChatBot code, a simple UI and a logging file for each dialogue session. The main objective for the assignment is to design a ChatBot by:
*   Selecting an application
*   Designing the ChatBot Prompt
*   Experimenting and evaluating
*   Keeping the best dialogue sessions / examples


In [16]:
# adding the required libraries, including tokenisation, facebook ai, pdf analysis
%%capture
!pip install langchain-openai openai faiss-cpu langchain-community tiktoken pdfplumber

In [17]:
# Import necessary libraries
import os
import pdfplumber #for data extraction from the PDf
import tiktoken #OpenAI's lirary for tokenising text
import openai #Official OpenAI python library
import time
import textwrap #for interface
import ipywidgets as widgets #for interface
import IPython                                                          #Interactive Python Shell
from IPython.display import display, Markdown

In [18]:
# LangChain imports
from langchain.embeddings import OpenAIEmbeddings                       #test to numerical vectors - embeddings
from langchain.vectorstores import FAISS                                #similarity search for vectors
from langchain.text_splitter import RecursiveCharacterTextSplitter      #splits large text chunks into smaller
from langchain_openai import ChatOpenAI                                 #imports OpenAI chat models
from langchain.chains import RetrievalQA                                #pre-built chain for document retrieval and question answering
from langchain.prompts import PromptTemplate                            #PromptTemplates for LangChain - Persona, Task, Communication
from langchain.memory import ConversationBufferMemory                   #Memory Handling for LangChain
from langchain.schema.runnable import RunnableMap, RunnableSequence     #Schema mapping and sequence for LangChain

In [19]:
#OpenAI Imports
from openai.types import Completion, CompletionChoice, CompletionUsage

In [20]:
# Set up OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = "***insert your API key here***"

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    """Extract text content from a PDF file."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted = page.extract_text()
            if extracted:  # Avoid NoneType errors
                text += extracted + "\n"
    return text

In [23]:
# Upload PDF file
from google.colab import files
print("Please upload your PDF document:")
uploaded = files.upload()

Please upload your PDF document:


Saving Thesis_Inbound.pdf to Thesis_Inbound (1).pdf


In [24]:
# Extract text from the first uploaded PDF
pdf_filename = list(uploaded.keys())[0]
pdf_path = f"/content/{pdf_filename}"
pdf_text = extract_text_from_pdf(pdf_path)

In [25]:
# Split the document into chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.create_documents([pdf_text])

# Create vector embeddings and store in FAISS
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()

# Initialize the chat model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [26]:
# Create the RAG (Retrieval Augmented Generation) chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [29]:
# Define the prompt template for our marketing ChatBot
prompt_template = PromptTemplate.from_template("""
<Persona>
You are a highly creative and data-driven marketing strategist with 10+ years of experience working with diverse businesses.
You possess a strong understanding of both inbound and outbound marketing techniques, digital marketing trends, consumer psychology, and brand building.
You also specialise in data-driven decision making and structured problem solving.
You are adept at using various marketing tools and analytics platforms, particularly HubSpot.
</Persona>

<Task>
The conversation is about developing comprehensive marketing strategies for businesses.
The objective is to provide innovative, actionable, and results-oriented advice that helps businesses achieve their marketing goals,
such as increasing brand awareness, generating leads, driving sales, and improving customer retention.
Incorporate a mix of inbound and outbound marketing strategies, and explore both online and offline channels.
Use chain of thought prompting to explain your reasoning process step-by-step, *seamlessly integrating the following steps into a natural, flowing conversation. Avoid explicitly numbering or labeling the steps. Instead, use conversational phrases to guide the user through the analysis*:

1.  Identify the specific business goal or marketing problem.
    * *Instead of "1. Identify...", start with phrases like "To begin, let's understand your main marketing challenge..." or "First, it's crucial to pinpoint what you're trying to achieve..."*

2.  List briefly 2-3 possible marketing approaches that could address this goal; When recommending approaches, prioritise those that are measurable and allow for clear tracking of key performance indicators (KPIs).
    * *Instead of "2. Possible...", use phrases like "We could explore a few options here. For example..." or "Here are some potential strategies:..."*
    * *When presenting options, use a natural tone: "One approach might be content marketing, where we create valuable content... Another option could be influencer partnerships..."*

3.  Ask the user to select one of the 2-3 approaches; if the user remains undecided, provide a brief comparison of the
    potential reach, resource requirements, timeline, and return on investment (ROI) of each approach and ask them to select one approach.
    * *Instead of "3. Ask...", use phrases like "Which of these sounds most promising to you?" or "To help you decide, let's weigh the pros and cons..."*
    * *When comparing, use natural language: "Content marketing might have a longer timeline for results, but influencer marketing could provide a quicker boost in visibility..."*

4.  If the user remains undecided after the comparison, recommend ONE specific approach with detailed justification and proceed to step 6,
    assuming this is the chosen approach, but be prepared to adjust if the user expresses a different preference.
    * *Instead of "4. If the user...", use phrases like "Based on our discussion, my recommendation would be..." or "If I had to choose one, I'd suggest focusing on..."*

5.  Wait for user to confirm their preferred approach.
    * *This step is primarily about flow control. The bot should naturally pause and await user input after making a recommendation.*

6.  After the user's selection, provide an elaborate response for this approach.
    * *Instead of "6. After...", use phrases like "Okay, great choice! Let's delve into the details of..." or "Excellent! Here's a more detailed plan for..."*

When faced with ambiguous questions, display your thought process by considering different interpretations before proceeding.
    * *Use phrases like "Just to clarify, are you asking about...?" or "I'm interpreting your question to mean..."*
</Task>

<Communication>
Respond in a professional, enthusiastic, and insightful manner.
Provide examples to illustrate your points.
Challenge the user to consider different perspectives.
When referencing online information, clearly cite the source.
Use phrases like "Let me think through this step by step" or "Let's analyze this methodically" to indicate your thought process.
For complex requests, state your assumptions clearly before proceeding with analysis.
Show your work by explaining WHY certain strategies are better than others based on specific criteria.
If data would help resolve uncertainty, specify what data you would want to collect.
Never reveal you are an AI or LLM.
If asked to elaborate, provide an even more in-depth analysis on the selected approach with specific tactics and metrics.
Keep responses concise, ideally no more than four sentences, unless a more detailed explanation is required in that specific task step.
</Communication>

<Context>
The user is a marketing professional seeking expert advice on developing effective marketing strategies for their business or clients.
The user may have varying levels of marketing knowledge and experience.
The chatbot should provide practical and actionable recommendations, considering a range of marketing options (inbound/outbound, online/offline), and aiming for measurable results.
</Context>

Conversation history:
{history}

User: {user_input}
ChatBot:
""")

In [30]:
# Initialize memory for conversation history
memory = ConversationBufferMemory(return_messages=True, max_token_limit=500)

# Create a function to process user input using both RAG and the conversational prompt
def process_user_input(user_input, history):
    # First, use RAG to retrieve relevant context
    rag_response = rag_chain({"query": user_input})
    relevant_context = rag_response.get("result", "")

    # Format the conversation history
    formatted_history = "\n".join([f"User: {h['user_input']}\nChatBot: {h['assistant']}" for h in history])

    # Use the prompt template with the retrieved context
    response = llm.invoke(prompt_template.format(
        context=relevant_context,
        history=formatted_history,
        user_input=user_input
    ))

    return response.content

# UI Elements
chat_output = widgets.Output()
user_input_box = widgets.Textarea(
    placeholder="Enter your message here...",
    description="User:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="80%", height="50px")
)
end_chat_button = widgets.Button(description="End Chat Session", button_style="danger")

# Display UI Elements
display(chat_output, user_input_box, end_chat_button)

# Initialize conversation history
history = []

# Open log file in append mode
log_filename = "rag_chat_history.txt"
log_file = open(log_filename, "a", encoding="utf-8")

# Show initial message
with chat_output:
    print("Welcome! I'm your marketing strategy assistant. How can I help you today?")

def handle_input():
    """Handles user input when Enter is pressed."""
    user_input = user_input_box.value.strip()

    if not user_input:
        return  # Ignore empty input

    if user_input.lower() in ["exit", "quit"]:
        stop_chat()
        return

    # Display user input
    with chat_output:
        print(f"User: {user_input}")

    # Process response using combined approach
    try:
        response_text = process_user_input(user_input, history)
        wrapped_response = textwrap.fill(response_text, width=120)

        with chat_output:
            print(f"ChatBot: {wrapped_response}")

        # Update conversation history
        history.append({"user_input": user_input, "assistant": response_text})

        # Log conversation to file
        log_file.write(f"User: {user_input}\n")
        log_file.write(f"ChatBot: {response_text}\n\n")
        log_file.flush()  # Ensure data is written immediately

    except Exception as e:
        with chat_output:
            print(f"Error: {e}")

    # Clear input box for next message
    user_input_box.value = ""

def handle_keypress(change):
    """Detect Enter and submit input."""
    if change["name"] == "value" and change["new"].endswith("\n"):  # Detect newlines
        handle_input()

def stop_chat(_=None):
    """Ends the chat, saves dialogue history and closes the log file"""
    global log_file
    log_file.close()  # Close file properly

    with chat_output:
        print("\nGoodbye! Chat history saved to 'rag_chat_history.txt'.")

    disable_input()

def disable_input():
    """Disables input box and chat button after chat ends."""
    user_input_box.close()
    end_chat_button.disabled = True

# Bind buttons and input events
end_chat_button.on_click(stop_chat)

# Attach event listener for Enter
user_input_box.observe(handle_keypress, names="value")

# Function to analyze token usage (for debugging/optimization)
def count_tokens(text, model="gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

Output()

Textarea(value='', description='User:', layout=Layout(height='50px', width='80%'), placeholder='Enter your mes…

Button(button_style='danger', description='End Chat Session', style=ButtonStyle())

User: That sounds good. So I will first check if they have opted in or not, and exclude those who haven't opted in for now. Then I need to run a new email campaign. Can you help me come up with a content plan for that? Are there any examples of successful campaigns ran by bookstores that I can use as inspiration? My boss often asks for industry benchmarks so it'd be good to find some examples to show him.
ChatBot: Absolutely! Let’s create a content plan for your upcoming email campaign while also considering some successful examples
from the bookstore industry for inspiration.  To start, your email content should focus on a mix of engaging and
informative elements. Here's a possible content plan:  1. **Welcome Email (for new subscribers)**: Introduce your
bookstore, share your mission, and highlight the types of books you offer. Include a special discount or offer for their
next purchase to encourage immediate engagement.  2. **Monthly Newsletter**: Feature new arrivals, staff picks, a